In [54]:
### Model G1 ####
SCHEME = 'Global'
FEATURES = ['Lat', 'Lon', 'Alt', 'Temp', 'Precip', 'Year', 'JulianDay_Sin']

### Import Libraries
# Base Libraries
import numpy as np
import pandas as pd
import re
# Tensorflow, scikit, kerasTuner
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, InputLayer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import RootMeanSquaredError, MeanAbsoluteError
from sklearn.preprocessing import MinMaxScaler
import keras_tuner as kt

In [55]:
# Function to import a dataset and transform headers for easier coding and convert Date column
# Pseudocode:
# 1. Import Dataset Save old Headers for later
# 2. Transform headernames to rid of units
# 3. Convert Date into Year and Julian Day
# 4. Peform a Sine Transformation on JulianDay
# 5. Return the dataset, and old headers
def importData(fileName):
    # Read in the correct file
    dataset = pd.read_csv(f'../../Data/{fileName}.csv')
    oldCols = list(dataset.columns)

    # Remove any units (anything in parentheses)
    codeCols = list(map(lambda x: re.sub(r'\(([^()]*)\)', '', x).strip(), oldCols))
    dataset.columns = codeCols

    # Transform Date into Year and JulianDay_Sin
    dataset['Date'] = pd.to_datetime(dataset['Date'], utc=True)
    dataset['Year'] = dataset['Date'].dt.year
    dataset['JulianDay'] = dataset['Date'].dt.dayofyear
    # Sine transformation to account for cyclical nature of Julian Day
    dataset['JulianDay_Sin'] = np.sin(2*np.pi*dataset['JulianDay']/365) 
    dataset.drop(columns=['Date', 'JulianDay'], inplace=True)
    
    #Add year and JulianDay_Sin to oldCols 
    oldCols.append('Year')
    oldCols.append('JulianDay_Sin')
    
    return dataset, oldCols

In [56]:
# Function that will sort dataset into scaled X and Y
# Pseudocode:
# 1. Separate dataset into Features and Target
# 2. Scale the Features array using MinMaxScaler
# 3. Return the scaled X, Y, and the scaler used
def scaleSplitData(dataset):
    # Separate Features and Target
    features = dataset[FEATURES]
    target = dataset[['O18', 'H2']]

    # Scale the Features
    scaler = MinMaxScaler()
    X = scaler.fit_transform(features.values)
    Y = target.values

    return X, Y, scaler

In [60]:
# Model Builder Function 
# Pseudocode:
# 1. Create a Sequential Model
# 2. Add an Input Layer
# 3. Prep the Search Space for Hyperparameter Tuning
# 4. Add a LSTM Layer with Hyperparameters
# 5. Add two Dense Layers with Hyperparameters 
# 6. Add a Dense Output Layer
# 7. Compile the Model with Hyperparameters
# 8. Return the Model
def modelBuilder(hp):
    # Create a Sequential Model
    model = Sequential()
    # Add an Input Layer
    model.add(InputLayer(input_shape=(len(FEATURES), 1)))

    # Prep the Search Space for Hyperparameter Tuning
    hp_numNeurons1 = hp.Int('numNeurons_LSTM', min_value=8, max_value=512, step=8)
    hp_numNeurons2 = hp.Int('numNeurons_Dense1', min_value=8, max_value=512, step=8)
    hp_numNeurons3 = hp.Int('numNeurons_Dense2', min_value=8, max_value=512, step=8)
    hp_lr = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    hp_batchSize = hp.Int('batch_size', min_value=8, max_value=512, step=8)

    # Add the hidden layers with Hyperparameters
    model.add(LSTM(units=hp_numNeurons1, activation='tanh', return_sequences=True))
    model.add(Dense(units=hp_numNeurons2, activation='relu'))
    model.add(Dense(units=hp_numNeurons3, activation='relu'))

    # Add the Output Layer
    model.add(Dense(2))

    # Compile the Model with Hyperparameters
    model.compile(optimizer=Adam(learning_rate=hp_lr), loss='mse', metrics=[RootMeanSquaredError(), MeanAbsoluteError()])

    return model

In [67]:
# Hyperparameter tuning process
# Pseudocode:
# 1. Create the Hyperband Tuner
# 2. Create a callback to stop training early
# 3. Perform the search
# 4. Get the best model hyperparameters
# 5. Return the best hyperparameters
def hyperParameterTuning(xTrain, yTrain):
    # Create the Hyperband Tuner
    tuner = kt.Hyperband(modelBuilder, objective='val_loss', max_epochs=100, factor=3, directory='Results', project_name='G1')

    # Create a callback to stop training early
    stop_early = EarlyStopping(monitor='val_loss', patience=5)

    # Perform the search
    tuner.search(X, Y, epochs=100, validation_split=0.2, callbacks=[stop_early])

    # Get the best model hyperparameters
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

    return best_hps

In [68]:
df = importData('DataTrain')[0]
X, Y, scaler = scaleSplitData(df)
model = hyperParameterTuning(X, Y)

Trial 2 Complete [00h 00m 02s]

Best val_loss So Far: None
Total elapsed time: 00h 00m 04s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
456               |96                |numNeurons_LSTM
376               |80                |numNeurons_Dense1
192               |168               |numNeurons_Dense2
0.01              |0.01              |learning_rate
304               |448               |batch_size
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
4                 |4                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2
Cause: Unable to locate the source code of <function Model.make_train_function.<locals>.train_function at 0x000001EDE36EFBE0>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the cod

Traceback (most recent call last):
  File "C:\Users\jaxton.gray\AppData\Roaming\Python\Python310\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "C:\Users\jaxton.gray\AppData\Roaming\Python\Python310\site-packages\keras_tuner\src\engine\base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "C:\Users\jaxton.gray\AppData\Roaming\Python\Python310\site-packages\keras_tuner\src\tuners\hyperband.py", line 427, in run_trial
    return super().run_trial(trial, *fit_args, **fit_kwargs)
  File "C:\Users\jaxton.gray\AppData\Roaming\Python\Python310\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "C:\Users\jaxton.gray\AppData\Roaming\Python\Python310\site-packages\keras_tuner\src\engine\tuner.py", line 233, in _build_and

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "C:\Users\jaxton.gray\AppData\Roaming\Python\Python310\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "C:\Users\jaxton.gray\AppData\Roaming\Python\Python310\site-packages\keras_tuner\src\engine\base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "C:\Users\jaxton.gray\AppData\Roaming\Python\Python310\site-packages\keras_tuner\src\tuners\hyperband.py", line 427, in run_trial
    return super().run_trial(trial, *fit_args, **fit_kwargs)
  File "C:\Users\jaxton.gray\AppData\Roaming\Python\Python310\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "C:\Users\jaxton.gray\AppData\Roaming\Python\Python310\s